# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:

# Write your code below.
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA") 

# Recursive search for parquet files
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
print("First 5 parquet files:", parquet_files[:5])  # preview first 5
#dd_px = dd.read_parquet(parquet_files).set_index("ticker")

First 5 parquet files: ['../../05_src/data/prices/MTN/MTN_2005/part.0.parquet', '../../05_src/data/prices/MTN/MTN_2005/part.1.parquet', '../../05_src/data/prices/MTN/MTN_2009/part.0.parquet', '../../05_src/data/prices/MTN/MTN_2009/part.1.parquet', '../../05_src/data/prices/MTN/MTN_2004/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [3]:
# Write your code below.

import dask.dataframe as dd

# Read into Dask DataFrame
ddf = dd.read_parquet(parquet_files)

# Feature engineering per ticker
dd_feat = (
    ddf
    .groupby("ticker", group_keys=False)
    .apply(lambda df: df.assign(
        Close_lag_1 = df["Close"].shift(1),
        Adj_Close_lag_1 = df["Adj Close"].shift(1),
        returns = (df["Close"] / df["Close"].shift(1)) - 1,
        hi_lo_range = df["High"] - df["Low"]
    ))
)



/tmp/ipykernel_107988/1973921302.py:12: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  .apply(lambda df: df.assign(
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [4]:
# Write your code below.

#Convert dask dataframe to pandas dataframe
df_feat = dd_feat.compute()

df_feat["returns_ma_10"] = (
    df_feat
    .groupby("ticker")["returns"]
    .transform(lambda x: x.rolling(window=10).mean())
)



/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:12

In [5]:
df_feat.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
188265,2007-07-05,17.510000,17.700001,16.809999,17.260000,8.360090,12300.0,UBP.csv,UBP,2007,NaN,NaN,NaN,0.890001,NaN
188266,2007-07-06,17.299999,17.650000,17.150000,17.650000,8.548994,11900.0,UBP.csv,UBP,2007,17.260000,8.360090,0.022596,0.500000,NaN
188267,2007-07-09,17.580000,17.620001,17.459999,17.590000,8.519932,6800.0,UBP.csv,UBP,2007,17.650000,8.548994,-0.003399,0.160002,NaN
188268,2007-07-10,17.420000,17.540001,17.350000,17.440001,8.447277,3200.0,UBP.csv,UBP,2007,17.590000,8.519932,-0.008528,0.190001,NaN
188269,2007-07-11,17.350000,17.940001,17.330000,17.350000,8.403684,9500.0,UBP.csv,UBP,2007,17.440001,8.447277,-0.005161,0.610001,NaN
188270,2007-07-12,17.250000,17.620001,17.250000,17.570000,8.510246,4900.0,UBP.csv,UBP,2007,17.350000,8.403684,0.012680,0.370001,NaN
188271,2007-07-13,17.450001,17.680000,17.440001,17.670000,8.558678,4500.0,UBP.csv,UBP,2007,17.570000,8.510246,0.005692,0.240000,NaN
188272,2007-07-16,17.500000,17.530001,17.410000,17.530001,8.490870,2200.0,UBP.csv,UBP,2007,17.670000,8.558678,-0.007923,0.120001,NaN
188273,2007-07-17,17.670000,17.709999,17.430000,17.639999,8.544149,12600.0,UBP.csv,UBP,2007,17.530001,8.490870,0.006275,0.279999,NaN
188274,2007-07-18,17.520000,17.660000,17.520000,17.660000,8.553836,3900.0,UBP.csv,UBP,2007,17.639999,8.544149,0.001134,0.139999,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Not 100 percent sure as I am still trying to understand everything. However, Dask is better to use when we have very large or complex datasets. It helps with processing as large datasets ordinarily will use lots of memory and take a longer time to process. The converse is true for Pandas. It is better for small datasets. You are able to get the results needed more quickly and it is doesn't drain memory.

It does seem like it would have been better to calculte it in dask. I do see performance issues in terms of speed with my computer running the display code above.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.